In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
# features: [Feature1, Feature2]
X = np.array([
    ["soleil", "chaud"],
    ["pluie",  "froid"],
    ["soleil", "froid"],
    ["pluie",  "chaud"]
])

# classes
y = np.array(["oui", "non", "oui", "non"])


In [32]:
X

array([['soleil', 'chaud'],
       ['pluie', 'froid'],
       ['soleil', 'froid'],
       ['pluie', 'chaud']], dtype='<U6')

In [33]:
y

array(['oui', 'non', 'oui', 'non'], dtype='<U3')

In [34]:
from collections import defaultdict

# 1. Probabilités des classes P(y)
classes, counts = np.unique(y, return_counts=True)
class_priors = {c: count / len(y) for c, count in zip(classes, counts)}

# 2. Probabilités conditionnelles P(x|y)
cond_probs = defaultdict(lambda: defaultdict(dict))

for c in classes:
    # sous-ensemble des lignes où la classe = c
    X_c = X[y == c]
    n_samples_c = len(X_c)

    # pour chaque colonne (feature)
    for j in range(X.shape[1]):
        values, val_counts = np.unique(X_c[:, j], return_counts=True)
        for v, vc in zip(values, val_counts):
            cond_probs[c][j][v] = vc / n_samples_c


In [36]:
def predict(x):
    scores = {}
    for c in classes:
        # log P(y)
        score = np.log(class_priors[c])
        # log P(x_i|y)
        for j, v in enumerate(x):
            if v in cond_probs[c][j]:
                score += np.log(cond_probs[c][j][v])
            else:
                # si valeur jamais vue -> proba = 0 (à améliorer avec lissage)
                score += -np.inf
        scores[c] = score
    # renvoie la classe avec le meilleur score
    return max(scores, key=scores.get)

# Exemple de prédiction
print(predict(["soleil", "froid"]))  # devrait donner "oui"
print(predict(["pluie", "froid"]))   # devrait donner "non"


oui
non
